### Подготовка признаков для градиентного бустинга

In [1]:
%load_ext autoreload
%autoreload 2

import gc
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns


import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score

import catboost as cb
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

# если у вас есть CUDA, то она понадобится там для экспериментов в catboost
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

sys.path.append('../')

В соревновании данные были предоставлены организаторами в формате `Parquet`.  Однако, для непосредственной работы с данными и построения моделей нам  прочитать их и трансформировать в pandas.DataFrame. При этом сделать это эффективно по памяти.

Для подготовки данных использовался baseline организаторов: читать данные итеративно небольшими чанками. Чанки организованы таким образом, что для конкретного клиента вся информация о его кредитной истории до момента подачи заявки на кредит расположена внутри одного чанка. Это позволяет загружать данные в память небольшими порциями, выделять все необходимые признаки и получать результирующий фрейм для моделирования. Для этих целей организаторы предоставили функцию `utils.read_parquet_dataset_from_local`.

Так же все чанки данных упорядочены по возрастанию даты заявки на кредит.


In [2]:
TRAIN_DATA_PATH = "../data/train_data/"
TEST_DATA_PATH = "../data/test_data/"

TRAIN_TARGET_PATH = "../data/train_target.csv"

In [3]:
from utils import read_parquet_dataset_from_local

В рамках данного соревнования участникам предстояло работать с кредитной историей клиентов. На основе кредитной истории клиента до момента подачи заявки на новый кредит нужно оценить, насколько благонадежным является клиент, и определить вероятность его ухода в дефолт по новому кредиту. Каждый кредит описывается набором из 60 категориальных признаков.


Базовым подходом к решению этой задачи является построение классических моделей машинного обучения на аггрегациях от последовательностей категориальных признаков. В baseline организаторов признаки были закодированы с помощью one-hot-encoding'а, к ним применено count-аггрегирование и на этом обучен градиентный бустинг из реализации `lightgbm`. 
Описанный подход к обработке кредитных историй клиентов реализован в виде класс-трансформера `CountAggregator` ниже.


Кроме этого, были подготовлены аналогичные признаки, но с применением весовой схемы для данных из кредитной истории - более старые кредиты вносили меньший вес, чем более поздние. Использовалась следующая схема:

$$ w_{i} = (\frac{i}{d})^\delta, $$

где $i$ - номер кредита в кредитной истории (поле "rn"), $d$ - общее количество кредитов в истории, $\delta$ - дополнительный параметр. В нашем случае в результате экспериментов $\delta = 1.4$.

Весовой подход к обработке кредитных историй клиентов реализован в виде класс-трансформера `WeightMeanAggregator` ниже.

In [4]:
TRAIN_FEATURES_PATH = "../data/train_features_gb/"
TEST_FEATURES_PATH = "../data/test_features_gb/"


In [5]:
! rm -r $TRAIN_FEATURES_PATH
! mkdir $TRAIN_FEATURES_PATH
! rm -r $TEST_FEATURES_PATH
! mkdir $TEST_FEATURES_PATH

"rm" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
Џ®¤Ї ЇЄ  Ё«Ё д ©« G:\Alfa_Bank_competition\data\train_features_gb г¦Ґ бгйҐбвўгҐв.
"rm" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
Џ®¤Ї ЇЄ  Ё«Ё д ©« G:\Alfa_Bank_competition\data\test_features_gb г¦Ґ бгйҐбвўгҐв.


In [6]:
class CountAggregator(object):
    
    def __init__(self):
        self.encoded_features = None
        
    def __extract_count_aggregations(self, data_frame: pd.DataFrame, mode: str) -> pd.DataFrame:
        # one-hot-encoding
        feature_columns = list(data_frame.columns.values)
        feature_columns.remove("id")
        feature_columns.remove("rn")

        dummies = pd.get_dummies(data_frame[feature_columns], columns=feature_columns)
        dummy_features = dummies.columns.values
        
        ohe_features = pd.concat([data_frame, dummies], axis=1)
        ohe_features = ohe_features.drop(columns=feature_columns)
        
        # count aggregation
        ohe_features.groupby("id")
        features = ohe_features.groupby("id")[dummy_features].sum().reset_index(drop=False)
        return features
        
    def __transform_data(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                                     mode: str = "fit_transform", save_to_path=None, verbose: bool=False):
        assert mode in ["fit_transform", "transform"], f"Unrecognized mode: {mode}! Please use one of the following modes: \"fit_transform\", \"transform\""
        preprocessed_frames = []
        for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once), 
                                       desc="Transforming sequential credit data"):
            data_frame = read_parquet_dataset_from_local(path_to_dataset, start_from=step, 
                                                         num_parts_to_read=num_parts_to_preprocess_at_once, 
                                                         verbose=verbose)
            features = self.__extract_count_aggregations(data_frame, mode=mode)
            if save_to_path:
                features.to_parquet(os.path.join(save_to_path, f"processed_chunk_{step}.pq"))
            preprocessed_frames.append(features)
        
        features = pd.concat(preprocessed_frames)
        features.fillna(np.uint8(0), inplace=True)
        dummy_features = list(features.columns.values)
        dummy_features.remove("id")
        if mode == "fit_transform":
            self.encoded_features = dummy_features
        else:
            assert not self.encoded_features is None, "Transformer not fitted"
            for col in self.encoded_features:
                if not col in dummy_features:
                    features[col] = np.uint8(0)
        return features[["id"]+self.encoded_features]
    
    def fit_transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int = 50,
                      save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="fit_transform",
                                     save_to_path=save_to_path, verbose=verbose)
    def transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                  save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="transform",
                                     save_to_path=save_to_path, verbose=verbose)

In [7]:
class WeightMeanAggregator(object):
    
    def __init__(self, alpha=1):
        self.encoded_features = None
        self.alpha = alpha
        
    def __extract_aggregations(self, data_frame: pd.DataFrame, mode: str) -> pd.DataFrame:
        # one-hot-encoding
        feature_columns = list(data_frame.columns.values)
        feature_columns.remove("id")
        feature_columns.remove("rn")

        dummies = pd.get_dummies(data_frame[feature_columns], columns=feature_columns)
        dummy_features = dummies.columns.values
        
        ohe_features = pd.concat([data_frame, dummies], axis=1)
        ohe_features = ohe_features.drop(columns=feature_columns)
        
        # lenght of history
        history_lenght = ohe_features.groupby('id')['rn'].max().reset_index(drop=False)
        history_lenght = history_lenght.rename(columns={'rn': 'history_lenght'})
        ohe_features = ohe_features.merge(history_lenght, on='id')

        ohe_features['weight'] = (ohe_features['rn'] / ohe_features['history_lenght'])**self.alpha
        sum_weight = ohe_features.groupby('id')['weight'].sum().reset_index(drop=False).rename(columns={'weight': 'sum_weight'})
        ohe_features = ohe_features.merge(sum_weight, on='id')
        
        generated_features = list(ohe_features.columns)
        generated_features.remove("id")
        generated_features.remove("rn")
        generated_features.remove("history_lenght")
        generated_features.remove("weight")
        generated_features.remove("sum_weight")
        
        for feature in generated_features:
            ohe_features[feature] = ohe_features[feature] * ohe_features['weight'] / ohe_features['sum_weight']
        
        # weight aggregation
        ohe_features.groupby("id")
        features = ohe_features.groupby("id")[generated_features].sum().reset_index(drop=False)
        features = features.merge(history_lenght, on='id')
        
        return features  
        
    def __transform_data(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                                     mode: str = "fit_transform", save_to_path=None, verbose: bool=False):
        assert mode in ["fit_transform", "transform"], f"Unrecognized mode: {mode}! Please use one of the following modes: \"fit_transform\", \"transform\""
        preprocessed_frames = []
        for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once), 
                                       desc="Transforming sequential credit data"):
            data_frame = read_parquet_dataset_from_local(path_to_dataset, start_from=step, 
                                                         num_parts_to_read=num_parts_to_preprocess_at_once, 
                                                         verbose=verbose)
            features = self.__extract_aggregations(data_frame, mode=mode)
            if save_to_path:
                features.to_parquet(os.path.join(save_to_path, f"processed_chunk_{step}.pq"))
            preprocessed_frames.append(features)
        
        features = pd.concat(preprocessed_frames)
        features.fillna(np.uint8(0), inplace=True)
        dummy_features = list(features.columns.values)
        dummy_features.remove("id")
        if mode == "fit_transform":
            self.encoded_features = dummy_features
        else:
            assert not self.encoded_features is None, "Transformer not fitted"
            for col in self.encoded_features:
                if not col in dummy_features:
                    features[col] = np.uint8(0)
        return features[["id"]+self.encoded_features]
    
    def fit_transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int = 50,
                      save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="fit_transform",
                                     save_to_path=save_to_path, verbose=verbose)
    def transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                  save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="transform",
                                     save_to_path=save_to_path, verbose=verbose)

In [8]:
aggregator_count = CountAggregator()
aggregator_weight_mean = WeightMeanAggregator(alpha=1.4)

In [9]:
%%time

train_data_weight_mean = aggregator_weight_mean.fit_transform(TRAIN_DATA_PATH, num_parts_to_preprocess_at_once=1, num_parts_total=12, 
                                      save_to_path=TRAIN_FEATURES_PATH, verbose=True)

Transforming sequential credit data:   0%|          | 0/12 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\train_data\train_data_0.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\train_data\train_data_1.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

...

Reading chunks:
G:\Alfa_Bank_competition\data_original\train_data\train_data_10.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\train_data\train_data_11.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Wall time: 10min 24s


In [10]:
train_data_weight_mean

,id,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,pre_since_opened_9,pre_since_opened_10,pre_since_opened_11,pre_since_opened_12,pre_since_opened_13,pre_since_opened_14,pre_since_opened_15,pre_since_opened_16,pre_since_opened_17,pre_since_opened_18,pre_since_opened_19,pre_since_confirmed_0,pre_since_confirmed_1,pre_since_confirmed_2,pre_since_confirmed_3,pre_since_confirmed_4,pre_since_confirmed_5,pre_since_confirmed_6,pre_since_confirmed_7,pre_since_confirmed_8,pre_since_confirmed_9,pre_since_confirmed_10,pre_since_confirmed_11,pre_since_confirmed_12,pre_since_confirmed_13,pre_since_confirmed_14,pre_since_confirmed_15,pre_since_confirmed_16,pre_since_confirmed_17,pre_pterm_0,pre_pterm_1,pre_pterm_2,pre_pterm_3,pre_pterm_4,pre_pterm_5,pre_pterm_6,pre_pterm_7,pre_pterm_8,pre_pterm_9,pre_pterm_10,pre_pterm_11,pre_pterm_12,pre_pterm_13,pre_pterm_14,pre_pterm_15,pre_pterm_16,pre_pterm_17,pre_fterm_0,pre_fterm_1,pre_fterm_2,pre_fterm_3,pre_fterm_4,pre_fterm_5,pre_fterm_6,pre_fterm_7,pre_fterm_8,pre_fterm_9,pre_fterm_10,pre_fterm_11,pre_fterm_12,pre_fterm_13,pre_fterm_14,pre_fterm_15,pre_fterm_16,pre_till_pclose_0,pre_till_pclose_1,pre_till_pclose_2,pre_till_pclose_3,pre_till_pclose_4,pre_till_pclose_5,pre_till_pclose_6,pre_till_pclose_7,pre_till_pclose_8,pre_till_pclose_9,pre_till_pclose_10,pre_till_pclose_11,pre_till_pclose_12,pre_till_pclose_13,pre_till_pclose_14,pre_till_pclose_15,pre_till_pclose_16,pre_till_fclose_0,pre_till_fclose_1,pre_till_fclose_2,pre_till_fclose_3,pre_till_fclose_4,pre_till_fclose_5,pre_till_fclose_6,pre_till_fclose_7,pre_till_fclose_8,pre_till_fclose_9,pre_till_fclose_10,pre_till_fclose_11,pre_till_fclose_12,pre_till_fclose_13,pre_till_fclose_14,pre_till_fclose_15,pre_loans_credit_limit_0,pre_loans_credit_limit_1,pre_loans_credit_limit_2,pre_loans_credit_limit_3,pre_loans_credit_limit_4,pre_loans_credit_limit_5,pre_loans_credit_limit_6,pre_loans_credit_limit_7,pre_loans_credit_limit_8,pre_loans_credit_limit_9,pre_loans_credit_limit_10,pre_loans_credit_limit_11,pre_loans_credit_limit_12,pre_loans_credit_limit_13,pre_loans_credit_limit_14,pre_loans_credit_limit_15,pre_loans_credit_limit_16,pre_loans_credit_limit_17,pre_loans_credit_limit_18,pre_loans_credit_limit_19,pre_loans_next_pay_summ_0,pre_loans_next_pay_summ_1,pre_loans_next_pay_summ_2,pre_loans_next_pay_summ_3,pre_loans_next_pay_summ_4,pre_loans_next_pay_summ_5,pre_loans_next_pay_summ_6,pre_loans_outstanding_1,pre_loans_outstanding_2,pre_loans_outstanding_3,pre_loans_outstanding_4,pre_loans_outstanding_5,pre_loans_total_overdue_0,pre_loans_max_overdue_sum_1,pre_loans_max_overdue_sum_2,pre_loans_max_overdue_sum_3,pre_loans_credit_cost_rate_0,pre_loans_credit_cost_rate_1,pre_loans_credit_cost_rate_2,pre_loans_credit_cost_rate_3,pre_loans_credit_cost_rate_4,pre_loans_credit_cost_rate_5,pre_loans_credit_cost_rate_6,pre_loans_credit_cost_rate_7,pre_loans_credit_cost_rate_8,pre_loans_credit_cost_rate_9,pre_loans_credit_cost_rate_10,pre_loans_credit_cost_rate_11,pre_loans_credit_cost_rate_12,pre_loans_credit_cost_rate_13,pre_loans5_0,pre_loans5_2,pre_loans5_3,pre_loans5_5,pre_loans5_6,pre_loans5_7,pre_loans5_13,pre_loans5_16,pre_loans530_0,pre_loans530_1,pre_loans530_2,pre_loans530_3,pre_loans530_4,pre_loans530_6,pre_loans530_7,pre_loans530_10,pre_loans530_11,pre_loans530_12,pre_loans530_13,pre_loans530_14,pre_loans530_15,pre_loans530_16,pre_loans530_18,pre_loans3060_2,pre_loans3060_5,pre_loans3060_7,pre_loans3060_8,pre_loans3060_9,pre_loans6090_1,pre_loans6090_2,pre_loans6090_4,pre_loans90_8,pre_loans90_13,pre_loans90_14,pre_loans90_19,is_zero_loans5_0,is_zero_loans5_1,is_zero_loans530_0,is_zero_loans530_1,is_zero_loans3060_0,is_zero_loans3060_1,is_zero_loans6090_0,is_zero_loans6090_1,is_zero_loans90_0,is_zero_loans90_1,pre_util_0,pre_util_1,pre_util_2,pre_util_3,pre_util_4,pre_util_5,pre_util_6,pre_util_7,pre_util_8,pre_util_9,pre_

In [11]:
train_data_count = aggregator_count.fit_transform(TRAIN_DATA_PATH, num_parts_to_preprocess_at_once=1, num_parts_total=12, 
                                      save_to_path=TRAIN_FEATURES_PATH, verbose=True)

Transforming sequential credit data:   0%|          | 0/12 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\train_data\train_data_0.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\train_data\train_data_1.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

...

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\train_data\train_data_10.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\train_data\train_data_11.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
train_data_count

,id,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,pre_since_opened_9,pre_since_opened_10,pre_since_opened_11,pre_since_opened_12,pre_since_opened_13,pre_since_opened_14,pre_since_opened_15,pre_since_opened_16,pre_since_opened_17,pre_since_opened_18,pre_since_opened_19,pre_since_confirmed_0,pre_since_confirmed_1,pre_since_confirmed_2,pre_since_confirmed_3,pre_since_confirmed_4,pre_since_confirmed_5,pre_since_confirmed_6,pre_since_confirmed_7,pre_since_confirmed_8,pre_since_confirmed_9,pre_since_confirmed_10,pre_since_confirmed_11,pre_since_confirmed_12,pre_since_confirmed_13,pre_since_confirmed_14,pre_since_confirmed_15,pre_since_confirmed_16,pre_since_confirmed_17,pre_pterm_0,pre_pterm_1,pre_pterm_2,pre_pterm_3,pre_pterm_4,pre_pterm_5,pre_pterm_6,pre_pterm_7,pre_pterm_8,pre_pterm_9,pre_pterm_10,pre_pterm_11,pre_pterm_12,pre_pterm_13,pre_pterm_14,pre_pterm_15,pre_pterm_16,pre_pterm_17,pre_fterm_0,pre_fterm_1,pre_fterm_2,pre_fterm_3,pre_fterm_4,pre_fterm_5,pre_fterm_6,pre_fterm_7,pre_fterm_8,pre_fterm_9,pre_fterm_10,pre_fterm_11,pre_fterm_12,pre_fterm_13,pre_fterm_14,pre_fterm_15,pre_fterm_16,pre_till_pclose_0,pre_till_pclose_1,pre_till_pclose_2,pre_till_pclose_3,pre_till_pclose_4,pre_till_pclose_5,pre_till_pclose_6,pre_till_pclose_7,pre_till_pclose_8,pre_till_pclose_9,pre_till_pclose_10,pre_till_pclose_11,pre_till_pclose_12,pre_till_pclose_13,pre_till_pclose_14,pre_till_pclose_15,pre_till_pclose_16,pre_till_fclose_0,pre_till_fclose_1,pre_till_fclose_2,pre_till_fclose_3,pre_till_fclose_4,pre_till_fclose_5,pre_till_fclose_6,pre_till_fclose_7,pre_till_fclose_8,pre_till_fclose_9,pre_till_fclose_10,pre_till_fclose_11,pre_till_fclose_12,pre_till_fclose_13,pre_till_fclose_14,pre_till_fclose_15,pre_loans_credit_limit_0,pre_loans_credit_limit_1,pre_loans_credit_limit_2,pre_loans_credit_limit_3,pre_loans_credit_limit_4,pre_loans_credit_limit_5,pre_loans_credit_limit_6,pre_loans_credit_limit_7,pre_loans_credit_limit_8,pre_loans_credit_limit_9,pre_loans_credit_limit_10,pre_loans_credit_limit_11,pre_loans_credit_limit_12,pre_loans_credit_limit_13,pre_loans_credit_limit_14,pre_loans_credit_limit_15,pre_loans_credit_limit_16,pre_loans_credit_limit_17,pre_loans_credit_limit_18,pre_loans_credit_limit_19,pre_loans_next_pay_summ_0,pre_loans_next_pay_summ_1,pre_loans_next_pay_summ_2,pre_loans_next_pay_summ_3,pre_loans_next_pay_summ_4,pre_loans_next_pay_summ_5,pre_loans_next_pay_summ_6,pre_loans_outstanding_1,pre_loans_outstanding_2,pre_loans_outstanding_3,pre_loans_outstanding_4,pre_loans_outstanding_5,pre_loans_total_overdue_0,pre_loans_max_overdue_sum_1,pre_loans_max_overdue_sum_2,pre_loans_max_overdue_sum_3,pre_loans_credit_cost_rate_0,pre_loans_credit_cost_rate_1,pre_loans_credit_cost_rate_2,pre_loans_credit_cost_rate_3,pre_loans_credit_cost_rate_4,pre_loans_credit_cost_rate_5,pre_loans_credit_cost_rate_6,pre_loans_credit_cost_rate_7,pre_loans_credit_cost_rate_8,pre_loans_credit_cost_rate_9,pre_loans_credit_cost_rate_10,pre_loans_credit_cost_rate_11,pre_loans_credit_cost_rate_12,pre_loans_credit_cost_rate_13,pre_loans5_0,pre_loans5_2,pre_loans5_3,pre_loans5_5,pre_loans5_6,pre_loans5_7,pre_loans5_13,pre_loans5_16,pre_loans530_0,pre_loans530_1,pre_loans530_2,pre_loans530_3,pre_loans530_4,pre_loans530_6,pre_loans530_7,pre_loans530_10,pre_loans530_11,pre_loans530_12,pre_loans530_13,pre_loans530_14,pre_loans530_15,pre_loans530_16,pre_loans530_18,pre_loans3060_2,pre_loans3060_5,pre_loans3060_7,pre_loans3060_8,pre_loans3060_9,pre_loans6090_1,pre_loans6090_2,pre_loans6090_4,pre_loans90_8,pre_loans90_13,pre_loans90_14,pre_loans90_19,is_zero_loans5_0,is_zero_loans5_1,is_zero_loans530_0,is_zero_loans530_1,is_zero_loans3060_0,is_zero_loans3060_1,is_zero_loans6090_0,is_zero_loans6090_1,is_zero_loans90_0,is_zero_loans90_1,pre_util_0,pre_util_1,pre_util_2,pre_util_3,pre_util_4,pre_util_5,pre_util_6,pre_util_7,pre_util_8,pre_util_9,pre_

Объединим простые аггрегационные и весовые аггрегационные признаки, а также добавим целевую метку ('flag')

In [13]:
train_target = pd.read_csv(TRAIN_TARGET_PATH)
train_target

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2999995,2999995,0
2999996,2999996,0
2999997,2999997,0
2999998,2999998,0


In [14]:
train_data = train_data_weight_mean.merge(train_data_count, on='id')
train_data = train_data.merge(train_target, on='id')
train_data

,id,pre_since_opened_0_x,pre_since_opened_1_x,pre_since_opened_2_x,pre_since_opened_3_x,pre_since_opened_4_x,pre_since_opened_5_x,pre_since_opened_6_x,pre_since_opened_7_x,pre_since_opened_8_x,pre_since_opened_9_x,pre_since_opened_10_x,pre_since_opened_11_x,pre_since_opened_12_x,pre_since_opened_13_x,pre_since_opened_14_x,pre_since_opened_15_x,pre_since_opened_16_x,pre_since_opened_17_x,pre_since_opened_18_x,pre_since_opened_19_x,pre_since_confirmed_0_x,pre_since_confirmed_1_x,pre_since_confirmed_2_x,pre_since_confirmed_3_x,pre_since_confirmed_4_x,pre_since_confirmed_5_x,pre_since_confirmed_6_x,pre_since_confirmed_7_x,pre_since_confirmed_8_x,pre_since_confirmed_9_x,pre_since_confirmed_10_x,pre_since_confirmed_11_x,pre_since_confirmed_12_x,pre_since_confirmed_13_x,pre_since_confirmed_14_x,pre_since_confirmed_15_x,pre_since_confirmed_16_x,pre_since_confirmed_17_x,pre_pterm_0_x,pre_pterm_1_x,pre_pterm_2_x,pre_pterm_3_x,pre_pterm_4_x,pre_pterm_5_x,pre_pterm_6_x,pre_pterm_7_x,pre_pterm_8_x,pre_pterm_9_x,pre_pterm_10_x,pre_pterm_11_x,pre_pterm_12_x,pre_pterm_13_x,pre_pterm_14_x,pre_pterm_15_x,pre_pterm_16_x,pre_pterm_17_x,pre_fterm_0_x,pre_fterm_1_x,pre_fterm_2_x,pre_fterm_3_x,pre_fterm_4_x,pre_fterm_5_x,pre_fterm_6_x,pre_fterm_7_x,pre_fterm_8_x,pre_fterm_9_x,pre_fterm_10_x,pre_fterm_11_x,pre_fterm_12_x,pre_fterm_13_x,pre_fterm_14_x,pre_fterm_15_x,pre_fterm_16_x,pre_till_pclose_0_x,pre_till_pclose_1_x,pre_till_pclose_2_x,pre_till_pclose_3_x,pre_till_pclose_4_x,pre_till_pclose_5_x,pre_till_pclose_6_x,pre_till_pclose_7_x,pre_till_pclose_8_x,pre_till_pclose_9_x,pre_till_pclose_10_x,pre_till_pclose_11_x,pre_till_pclose_12_x,pre_till_pclose_13_x,pre_till_pclose_14_x,pre_till_pclose_15_x,pre_till_pclose_16_x,pre_till_fclose_0_x,pre_till_fclose_1_x,pre_till_fclose_2_x,pre_till_fclose_3_x,pre_till_fclose_4_x,pre_till_fclose_5_x,pre_till_fclose_6_x,pre_till_fclose_7_x,pre_till_fclose_8_x,pre_till_fclose_9_x,pre_till_fclose_10_x,pre_till_fclose_11_x,pre_till_fclose_12_x,pre_till_fclose_13_x,pre_till_fclose_14_x,pre_till_fclose_15_x,pre_loans_credit_limit_0_x,pre_loans_credit_limit_1_x,pre_loans_credit_limit_2_x,pre_loans_credit_limit_3_x,pre_loans_credit_limit_4_x,pre_loans_credit_limit_5_x,pre_loans_credit_limit_6_x,pre_loans_credit_limit_7_x,pre_loans_credit_limit_8_x,pre_loans_credit_limit_9_x,pre_loans_credit_limit_10_x,pre_loans_credit_limit_11_x,pre_loans_credit_limit_12_x,pre_loans_credit_limit_13_x,pre_loans_credit_limit_14_x,pre_loans_credit_limit_15_x,pre_loans_credit_limit_16_x,pre_loans_credit_limit_17_x,pre_loans_credit_limit_18_x,pre_loans_credit_limit_19_x,pre_loans_next_pay_summ_0_x,pre_loans_next_pay_summ_1_x,pre_loans_next_pay_summ_2_x,pre_loans_next_pay_summ_3_x,pre_loans_next_pay_summ_4_x,pre_loans_next_pay_summ_5_x,pre_loans_next_pay_summ_6_x,pre_loans_outstanding_1_x,pre_loans_outstanding_2_x,pre_loans_outstanding_3_x,pre_loans_outstanding_4_x,pre_loans_outstanding_5_x,pre_loans_total_overdue_0_x,pre_loans_max_overdue_sum_1_x,pre_loans_max_overdue_sum_2_x,pre_loans_max_overdue_sum_3_x,pre_loans_credit_cost_rate_0_x,pre_loans_credit_cost_rate_1_x,pre_loans_credit_cost_rate_2_x,pre_loans_credit_cost_rate_3_x,pre_loans_credit_cost_rate_4_x,pre_loans_credit_cost_rate_5_x,pre_loans_credit_cost_rate_6_x,pre_loans_credit_cost_rate_7_x,pre_loans_credit_cost_rate_8_x,pre_loans_credit_cost_rate_9_x,pre_loans_credit_cost_rate_10_x,pre_loans_credit_cost_rate_11_x,pre_loans_credit_cost_rate_12_x,pre_loans_credit_cost_rate_13_x,pre_loans5_0_x,pre_loans5_2_x,pre_loans5_3_x,pre_loans5_5_x,pre_loans5_6_x,pre_loans5_7_x,pre_loans5_13_x,pre_loans5_16_x,pre_loans530_0_x,pre_loans530_1_x,pre_loans530_2_x,pre_loans530_3_x,pre_loans530_4_x,pre_loans530_6_x,pre_loans530_7_x,pre_loans530_10_x,pre_loans530_11_x,pre_loans530_12_x,pre_loans530_13_x,pre_loans530_14_x,pre_loans530_15_x,pre_loans530_16_x,pre_loans530_18_x,pre_loans3060_2_x,pre_loans3060_5_x,pre_loans3060_7_x,pre_loans3060_8_x,pre_loans3060_9_x,pre_loans6090_1_x,pre_loans6090_2_x,pre_l

In [16]:
train_data.to_csv('../boosting_data_weight_mean_count.csv')

Аналогичным образом извлечем признаки для заявок из тестовой выборки:

In [17]:
test_data_weight_mean = aggregator_weight_mean.transform(TEST_DATA_PATH, num_parts_to_preprocess_at_once=1, num_parts_total=2,
                                                         save_to_path=TEST_FEATURES_PATH, verbose=True)

test_data_count = aggregator_count.transform(TEST_DATA_PATH, num_parts_to_preprocess_at_once=1, num_parts_total=2,
                                             save_to_path=TEST_FEATURES_PATH, verbose=True)

test_data = test_data_weight_mean.merge(test_data_count, on='id')

Transforming sequential credit data:   0%|          | 0/2 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\test_data\test_data_0.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\test_data\test_data_1.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Transforming sequential credit data:   0%|          | 0/2 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\test_data\test_data_0.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
G:\Alfa_Bank_competition\data_original\test_data\test_data_1.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
test_data

,id,pre_since_opened_0_x,pre_since_opened_1_x,pre_since_opened_2_x,pre_since_opened_3_x,pre_since_opened_4_x,pre_since_opened_5_x,pre_since_opened_6_x,pre_since_opened_7_x,pre_since_opened_8_x,pre_since_opened_9_x,pre_since_opened_10_x,pre_since_opened_11_x,pre_since_opened_12_x,pre_since_opened_13_x,pre_since_opened_14_x,pre_since_opened_15_x,pre_since_opened_16_x,pre_since_opened_17_x,pre_since_opened_18_x,pre_since_opened_19_x,pre_since_confirmed_0_x,pre_since_confirmed_1_x,pre_since_confirmed_2_x,pre_since_confirmed_3_x,pre_since_confirmed_4_x,pre_since_confirmed_5_x,pre_since_confirmed_6_x,pre_since_confirmed_7_x,pre_since_confirmed_8_x,pre_since_confirmed_9_x,pre_since_confirmed_10_x,pre_since_confirmed_11_x,pre_since_confirmed_12_x,pre_since_confirmed_13_x,pre_since_confirmed_14_x,pre_since_confirmed_15_x,pre_since_confirmed_16_x,pre_since_confirmed_17_x,pre_pterm_0_x,pre_pterm_1_x,pre_pterm_2_x,pre_pterm_3_x,pre_pterm_4_x,pre_pterm_5_x,pre_pterm_6_x,pre_pterm_7_x,pre_pterm_8_x,pre_pterm_9_x,pre_pterm_10_x,pre_pterm_11_x,pre_pterm_12_x,pre_pterm_13_x,pre_pterm_14_x,pre_pterm_15_x,pre_pterm_16_x,pre_pterm_17_x,pre_fterm_0_x,pre_fterm_1_x,pre_fterm_2_x,pre_fterm_3_x,pre_fterm_4_x,pre_fterm_5_x,pre_fterm_6_x,pre_fterm_7_x,pre_fterm_8_x,pre_fterm_9_x,pre_fterm_10_x,pre_fterm_11_x,pre_fterm_12_x,pre_fterm_13_x,pre_fterm_14_x,pre_fterm_15_x,pre_fterm_16_x,pre_till_pclose_0_x,pre_till_pclose_1_x,pre_till_pclose_2_x,pre_till_pclose_3_x,pre_till_pclose_4_x,pre_till_pclose_5_x,pre_till_pclose_6_x,pre_till_pclose_7_x,pre_till_pclose_8_x,pre_till_pclose_9_x,pre_till_pclose_10_x,pre_till_pclose_11_x,pre_till_pclose_12_x,pre_till_pclose_13_x,pre_till_pclose_14_x,pre_till_pclose_15_x,pre_till_pclose_16_x,pre_till_fclose_0_x,pre_till_fclose_1_x,pre_till_fclose_2_x,pre_till_fclose_3_x,pre_till_fclose_4_x,pre_till_fclose_5_x,pre_till_fclose_6_x,pre_till_fclose_7_x,pre_till_fclose_8_x,pre_till_fclose_9_x,pre_till_fclose_10_x,pre_till_fclose_11_x,pre_till_fclose_12_x,pre_till_fclose_13_x,pre_till_fclose_14_x,pre_till_fclose_15_x,pre_loans_credit_limit_0_x,pre_loans_credit_limit_1_x,pre_loans_credit_limit_2_x,pre_loans_credit_limit_3_x,pre_loans_credit_limit_4_x,pre_loans_credit_limit_5_x,pre_loans_credit_limit_6_x,pre_loans_credit_limit_7_x,pre_loans_credit_limit_8_x,pre_loans_credit_limit_9_x,pre_loans_credit_limit_10_x,pre_loans_credit_limit_11_x,pre_loans_credit_limit_12_x,pre_loans_credit_limit_13_x,pre_loans_credit_limit_14_x,pre_loans_credit_limit_15_x,pre_loans_credit_limit_16_x,pre_loans_credit_limit_17_x,pre_loans_credit_limit_18_x,pre_loans_credit_limit_19_x,pre_loans_next_pay_summ_0_x,pre_loans_next_pay_summ_1_x,pre_loans_next_pay_summ_2_x,pre_loans_next_pay_summ_3_x,pre_loans_next_pay_summ_4_x,pre_loans_next_pay_summ_5_x,pre_loans_next_pay_summ_6_x,pre_loans_outstanding_1_x,pre_loans_outstanding_2_x,pre_loans_outstanding_3_x,pre_loans_outstanding_4_x,pre_loans_outstanding_5_x,pre_loans_total_overdue_0_x,pre_loans_max_overdue_sum_1_x,pre_loans_max_overdue_sum_2_x,pre_loans_max_overdue_sum_3_x,pre_loans_credit_cost_rate_0_x,pre_loans_credit_cost_rate_1_x,pre_loans_credit_cost_rate_2_x,pre_loans_credit_cost_rate_3_x,pre_loans_credit_cost_rate_4_x,pre_loans_credit_cost_rate_5_x,pre_loans_credit_cost_rate_6_x,pre_loans_credit_cost_rate_7_x,pre_loans_credit_cost_rate_8_x,pre_loans_credit_cost_rate_9_x,pre_loans_credit_cost_rate_10_x,pre_loans_credit_cost_rate_11_x,pre_loans_credit_cost_rate_12_x,pre_loans_credit_cost_rate_13_x,pre_loans5_0_x,pre_loans5_2_x,pre_loans5_3_x,pre_loans5_5_x,pre_loans5_6_x,pre_loans5_7_x,pre_loans5_13_x,pre_loans5_16_x,pre_loans530_0_x,pre_loans530_1_x,pre_loans530_2_x,pre_loans530_3_x,pre_loans530_4_x,pre_loans530_6_x,pre_loans530_7_x,pre_loans530_10_x,pre_loans530_11_x,pre_loans530_12_x,pre_loans530_13_x,pre_loans530_14_x,pre_loans530_15_x,pre_loans530_16_x,pre_loans530_18_x,pre_loans3060_2_x,pre_loans3060_5_x,pre_loans3060_7_x,pre_loans3060_8_x,pre_loans3060_9_x,pre_loans6090_1_x,pre_loans6090_2_x,pre_l

In [ ]:
test_data.to_csv('../boosting_data_weight_mean_count_test.csv')